# Testing Lisence Plates

### Imports

In [1]:
import numpy as np
import cv2
from os import listdir
READ_IMG_PATH = '/home/fizzer/Insync/laura.romer@gmail.com/Google Drive - Shared with me/enph353_competition/assets/images/outer_lap/Liscence_plate/' 

In [2]:

# Load the image names
imgs = listdir(READ_IMG_PATH)


# Load the image
img = cv2.imread(READ_IMG_PATH+imgs[20])

cv2.imshow('',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:


# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow('', gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Apply thresholding to the grayscale image
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
useful_contours = [contour for contour in contours if cv2.contourArea(contour) > 5]




In [10]:
drawn = cv2.drawContours(img, useful_contours, -1, (0, 255, 0), 3)
cv2.imshow('',drawn)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:

# Loop over the contours
for contour in useful_contours:

    # Approximate the contour as a polygon
    approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)

    # If the polygon has four sides and is convex
    if len(approx) == 4 and cv2.isContourConvex(approx) and cv2.contourArea(contour) > 5:

        # Compute the bounding box of the polygon
        x, y, w, h = cv2.boundingRect(approx)

        # If the bounding box is roughly the aspect ratio of a license plate
        aspect_ratio = w / h
        if 1.0 <= aspect_ratio <= 5.0:

            # Draw a rectangle around the polygon
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

            # Define the corners of the plate region
            corners_src = np.float32([[x, y], [x + w, y], [x, y + h], [x + w, y + h]])

            # Define the corners of the output image (assumes a fixed size)
            corners_dst = np.float32([[0, 0], [500, 0], [0, 200], [500, 200]])

            # Compute the perspective transform matrix
            M = cv2.getPerspectiveTransform(corners_src, corners_dst)

            # Apply the perspective transform
            warped = cv2.warpPerspective(img, M, (500, 200))

            # Display the transformed image
            cv2.imshow('License Plate', warped)
            cv2.waitKey(0)

# Display the original image with the detected plates
cv2.imshow('License Plates', img)
cv2.waitKey(0)
cv2.destroyAllWindows()